현재 적용하고자 하는 게이밍 알고리즘
박현호의 깃허브 참조
게임을 위한 ai 3장
## 3.3.3 찾기, 도망가기
- 찾기는 캐릭터의 위치를 타깃 위치로 매칭한다.
	- 키네마틱 찾기 알고리즘은 타깃으로 방향을 알아내고 가능한 빠르게 해당 방향으로 향한다
		- 조종 행동의 출력 값이 가속도이기 때문에 가능한 한 가속을 한다
	- 대부분에 경우 최대 속도가 존재하기에 최대값은 명확하게 설정된다
		- 변수 또는 상숫값으로 존재한다
	- 현재 캐릭터의 속도(속도 벡터의 길이)는 지속적으로 체크되고 최대 속도를 넘어가면 최대 속도에 한정된다
		- 이것은 보통 업데이트 함수의 후처리 과정에서 이뤄짐
		- 조종 행동에서 처리하지는 않음

In [9]:
import math
from flask import Flask, request, jsonify
import os
import torch

class Vector:
    def __init__(self, x=0.0, y=0.0):
        self.x = x
        self.y = y

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

    def length(self):
        return math.sqrt(self.x**2 + self.y**2)

    def normalize(self):
        length = self.length()
        if length > 0:
            self.x /= length
            self.y /= length
        return self

    def __str__(self):
        return f"Vector({self.x:.2f}, {self.y:.2f})"

class Kinematic:
    def __init__(self, position=Vector(), orientation=0.0, velocity=Vector(), rotation=0.0):
        self.position = position
        self.orientation = orientation
        self.velocity = velocity
        self.rotation = rotation

    def update(self, steering, maxSpeed, time):
        self.velocity = self.velocity + (steering.linear * time)
        self.rotation += steering.angular * time

        if self.velocity.length() > maxSpeed:
            self.velocity.normalize()
            self.velocity *= maxSpeed

        self.position = self.position + (self.velocity * time)
        self.orientation += self.rotation * time

    def asVector(self):
        return Vector(math.cos(self.orientation), math.sin(self.orientation))

def newOrientation(current, velocity):
    if velocity.length() > 0:
        return math.atan2(velocity.x, velocity.y)
    return current
    
class KinematicSteeringOutput():
    def __init__(self, velocity=Vector(), rotation=0.0):
        self.velocity = velocity
        self.rotation = rotation

class KinematicSteeringBehavior:
    def __init__(self, character, maxSpeed):
        self.character = character
        self.maxSpeed = maxSpeed

    def getSteering(self):
        raise NotImplementedError("서브클래스는 getSteering을 구현해야 합니다.")

class KinematicArrive(KinematicSteeringBehavior):
    def __init__(self, character, target, maxSpeed, radius, timeToTarget=0.25):
        super().__init__(character, maxSpeed)
        self.target = target
        self.radius = radius
        self.timeToTarget = timeToTarget

    def getSteering(self):
        result = KinematicSteeringOutput()
        result.velocity = self.target.position - self.character.position
        if result.velocity.length() < self.radius:
            return None
        result.velocity = result.velocity * (1 / self.timeToTarget)
        if result.velocity.length() > self.maxSpeed:
            result.velocity.normalize()
            result.velocity = result.velocity * self.maxSpeed
        self.character.orientation = newOrientation(self.character.orientation, result.velocity)
        result.rotation = 0
        return result

# 동적 스티어링 출력 클래스
class SteeringOutput():
    def __init__(self, linear=Vector(), angular=0.0):
        self.linear = linear
        self.angular = angular

class DynamicSteeringBehavior:
    def __init__(self, character, maxAcceleration):
        self.character = character
        self.maxAcceleration = maxAcceleration

    def getSteering(self):
        raise NotImplementedError("서브클래스는 getSteering을 구현해야 합니다.")
          
# 추적 행동 (Seek) - 동적 스티어링
class Seek(DynamicSteeringBehavior):
    def __init__(self, character, target, maxAcceleration):
        super().__init__(character, maxAcceleration)
        self.target = target

    def getSteering(self):
        result = SteeringOutput()
        direction = self.target.position - self.character.position
        direction.normalize()
        result.linear = direction * self.maxAcceleration
        result.angular = 0
        return result
        
if __name__ == "__main__":
    character = Kinematic(position=Vector(0, 0), orientation=0.0)
    target = Kinematic(position=Vector(5, 5), orientation=0.0)
    seek = Seek(character, target, maxAcceleration=1.0)
    steering = seek.getSteering()
    print("동적 Seek 행동:")
    print(f"선형 가속도: {steering.linear}, 각가속도: {steering.angular:.2f}")
    character.update(steering, maxSpeed=2.0, time=1.0)
    print(f"업데이트 후 위치: {character.position}, 속도: {character.velocity}, 방향: {character.orientation:.2f}")

동적 Seek 행동:
선형 가속도: Vector(0.71, 0.71), 각가속도: 0.00
업데이트 후 위치: Vector(0.71, 0.71), 속도: Vector(0.71, 0.71), 방향: 0.00


- 기존의 모든 클래스를 명시해 가독성이 떨어지는 문제를 해결하기 위한 모듈화 진행
- 클래스를 모델에서 불러오는 방식으로 진행했으며 위 게임 알고리즘은 2차원 평면에서 적용가능한 것
- 따라서 시뮬레이션 상의 환경을 $xz$ plane 만을 고려하고 $y$ 축은 고려하지 않겠다

In [11]:
import math
from flask import Flask, request, jsonify
import os
import torch
from gameAI import Kinematic, Vector, Kinematic, Seek

if __name__ == "__main__":
    character = Kinematic(position=Vector(0, 0), orientation=0.0)
    target = Kinematic(position=Vector(5, 5), orientation=0.0)
    seek = Seek(character, target, maxAcceleration=1.0)
    steering = seek.getSteering()
    print("동적 Seek 행동:")
    print(f"선형 가속도: {steering.linear}, 각가속도: {steering.angular:.2f}")
    character.update(steering, maxSpeed=2.0, time=1.0)
    print(f"업데이트 후 위치: {character.position}, 속도: {character.velocity}, 방향: {character.orientation:.2f}")

동적 Seek 행동:
선형 가속도: Vector(0.71, 0.71), 각가속도: 0.00
업데이트 후 위치: Vector(0.71, 0.71), 속도: Vector(0.71, 0.71), 방향: 0.00


In [48]:
import math
from flask import Flask, request, jsonify
import os
import torch
from gameAI import Kinematic, Vector, Kinematic, Seek
from utils import process_info_data

app = Flask(__name__)

@app.route('/info', methods=['POST'])
def info():
    data = request.get_json(force=True)
    if not data:
        return jsonify({"error": "No JSON received"}), 400
    app.logger.info("Received /info data: %s", data)
    return jsonify({"status": "success", "message": "Data received"}), 200

if __name__ == "__main__":
    character = Kinematic(position=Vector(0, 0), orientation=0.0)
    target = Kinematic(position=Vector(5, 5), orientation=0.0)
    seek = Seek(character, target, maxAcceleration=1.0)
    steering = seek.getSteering()
    print("동적 Seek 행동:")
    print(f"선형 가속도: {steering.linear}, 각가속도: {steering.angular:.2f}")
    character.update(steering, maxSpeed=2.0, time=1.0)
    print(f"업데이트 후 위치: {character.position}, 속도: {character.velocity}, 방향: {character.orientation:.2f}")
    app.run(host='0.0.0.0', port=5000)


동적 Seek 행동:
선형 가속도: Vector(0.71, 0.71), 각가속도: 0.00
업데이트 후 위치: Vector(0.71, 0.71), 속도: Vector(0.71, 0.71), 방향: 0.00
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.124:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Apr/2025 15:53:30] "POST /info HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2025 15:53:30] "POST /info HTTP/1.1" 200 -


Received /info data: {'time': 1251.7657470703125, 'distance': 258.11614990234375, 'playerPos': {'x': 59.349998474121094, 'y': 7.991199493408203, 'z': 30.230897903442383}, 'playerSpeed': 0.01874319650232792, 'playerHealth': 100.0, 'playerTurretX': 2.9181717042803257e-09, 'playerTurretY': 0.0, 'playerBodyX': 2.9181717042803257e-09, 'playerBodyY': -3.256887814995224e-11, 'enemyPos': {'x': 135.46002197265625, 'y': 8.597223281860352, 'z': 276.8699951171875}, 'enemySpeed': 0.002733619185164571, 'enemyHealth': 100.0, 'enemyTurretX': 180.0012969970703, 'enemyTurretY': 0.0, 'enemyBodyX': 180.0012969970703, 'enemyBodyY': -7.34899913368281e-07}
Received /info data: {'time': 1250.7564697265625, 'distance': 258.1162109375, 'playerPos': {'x': 59.349998474121094, 'y': 7.972282409667969, 'z': 30.230897903442383}, 'playerSpeed': 0.002884102752432227, 'playerHealth': 100.0, 'playerTurretX': 2.9181717042803257e-09, 'playerTurretY': 0.0, 'playerBodyX': 2.9181717042803257e-09, 'playerBodyY': -3.25688781499

127.0.0.1 - - [09/Apr/2025 15:53:30] "POST /info HTTP/1.1" 200 -


Received /info data: {'time': 1252.7701416015625, 'distance': 258.1161804199219, 'playerPos': {'x': 59.349998474121094, 'y': 7.983585357666016, 'z': 30.230897903442383}, 'playerSpeed': 0.00758082140237093, 'playerHealth': 100.0, 'playerTurretX': 2.9181717042803257e-09, 'playerTurretY': 0.0, 'playerBodyX': 2.9181717042803257e-09, 'playerBodyY': -3.256887814995224e-11, 'enemyPos': {'x': 135.46002197265625, 'y': 8.599906921386719, 'z': 276.8699951171875}, 'enemySpeed': 0.002671897877007723, 'enemyHealth': 100.0, 'enemyTurretX': 180.0012969970703, 'enemyTurretY': -1.921010607475182e-06, 'enemyBodyX': 180.0012969970703, 'enemyBodyY': 1.921010607475182e-06}


...

Received /info data: {'time': 1255.7886962890625, 'distance': 258.1161804199219, 'playerPos': {'x': 59.34999465942383, 'y': 7.986023426055908, 'z': 30.23089599609375}, 'playerSpeed': 0.01684260554611683, 'playerHealth': 100.0, 'playerTurretX': 2.9181717042803257e-09, 'playerTurretY': 0.0, 'playerBodyX': 2.9181717042803257e-09, 'playerBodyY': -3.256887814995224e-11, 'enemyPos': {'x': 135.46002197265625, 'y': 8.599992752075195, 'z': 276.8699951171875}, 'enemySpeed': 3.7755226003355347e-06, 'enemyHealth': 100.0, 'enemyTurretX': 180.0012969970703, 'enemyTurretY': -3.935278982680757e-06, 'enemyBodyX': 180.0012969970703, 'enemyBodyY': 3.935278982680757e-06}


127.0.0.1 - - [09/Apr/2025 15:53:34] "POST /info HTTP/1.1" 200 -


Received /info data: {'time': 1256.789306640625, 'distance': 258.1161804199219, 'playerPos': {'x': 59.349998474121094, 'y': 7.984996795654297, 'z': 30.23089599609375}, 'playerSpeed': 0.0010260113049298525, 'playerHealth': 100.0, 'playerTurretX': 2.9181717042803257e-09, 'playerTurretY': 0.0, 'playerBodyX': 2.9181717042803257e-09, 'playerBodyY': -3.256887814995224e-11, 'enemyPos': {'x': 135.46002197265625, 'y': 8.598196029663086, 'z': 276.8699951171875}, 'enemySpeed': 0.0017956264782696962, 'enemyHealth': 100.0, 'enemyTurretX': 180.0012969970703, 'enemyTurretY': -3.426167268116842e-06, 'enemyBodyX': 180.0012969970703, 'enemyBodyY': 3.426167268116842e-06}


127.0.0.1 - - [09/Apr/2025 15:53:35] "POST /info HTTP/1.1" 200 -


Received /info data: {'time': 1257.8018798828125, 'distance': 258.11614990234375, 'playerPos': {'x': 59.35000228881836, 'y': 7.998065948486328, 'z': 30.230899810791016}, 'playerSpeed': 0.012906872667372227, 'playerHealth': 100.0, 'playerTurretX': 2.9181717042803257e-09, 'playerTurretY': 0.0, 'playerBodyX': 2.9181717042803257e-09, 'playerBodyY': -3.256887814995224e-11, 'enemyPos': {'x': 135.46002197265625, 'y': 8.599991798400879, 'z': 276.8699951171875}, 'enemySpeed': 0.0017734704306349158, 'enemyHealth': 100.0, 'enemyTurretX': 180.0012969970703, 'enemyTurretY': 0.0, 'enemyBodyX': 180.0012969970703, 'enemyBodyY': -3.4150227747886674e-06}


In [62]:
# server.py
import math
from shared_data import shared
import time

def main():
    print("Shared object in server.py:", shared)  # 공유 객체 확인
    while True:
        data = shared.get_data()
        if data is not None:
            print("Data from server.py:", data)
        else:
            print("No data available in server.py yet.")
        time.sleep(2)

if __name__ == "__main__":
    main()

Shared object in server.py: <shared_data.SharedData object at 0x0000019F9ECB9AC0>
No data available in server.py yet.
No data available in server.py yet.


KeyboardInterrupt: 

In [ ]:
import time, requests, math
from gameAI import Kinematic, Vector, Kinematic, Seek

def main():
    while True:
        try:
            response = requests.get("http://localhost:5000/get_data")
            if response.status_code == 200:
                data = response.json()["data"]
                print("Data from server.py:", data)

                # 데이터를 변수로 추출
                time_value = data["time"]
                distance = data["distance"]
                
                player_pos_x = data["playerPos"]["x"]
                player_pos_y = data["playerPos"]["y"]
                player_pos_z = data["playerPos"]["z"]
                player_speed = data["playerSpeed"]
                player_health = data["playerHealth"]
                player_turret_x = data["playerTurretX"]
                player_turret_y = data["playerTurretY"]
                player_body_x = data["playerBodyX"]
                player_body_y = data["playerBodyY"]
                
                enemy_pos_x = data["enemyPos"]["x"]
                enemy_pos_y = data["enemyPos"]["y"]
                enemy_pos_z = data["enemyPos"]["z"]
                enemy_speed = data["enemySpeed"]
                enemy_health = data["enemyHealth"]
                enemy_turret_x = data["enemyTurretX"]
                enemy_turret_y = data["enemyTurretY"]
                enemy_body_x = data["enemyBodyX"]
                enemy_body_y = data["enemyBodyY"]

                print(f"Time: {time_value}")
                print(f"Distance: {distance}")
                print(f"Player Position: ({player_pos_x}, {player_pos_y}, {player_pos_z})")
                print(f"Enemy Position: ({enemy_pos_x}, {enemy_pos_y}, {enemy_pos_z})")
                print(f"Player Health: {player_health}, Enemy Health: {enemy_health}")

                # 거리 계산 확인
                calculated_distance = math.sqrt(
                    (player_pos_x - enemy_pos_x) ** 2 +
                    (player_pos_z - enemy_pos_z) ** 2
                )
                print(f"Calculated Distance: {calculated_distance}")

            else:
                print("No data available in server.py yet.")
        except requests.exceptions.RequestException as e:
            print("Error fetching data:", e)
        time.sleep(2)

if __name__ == "__main__":
    main()

In [ ]:
import time, requests, math
from gameAI import Kinematic, Vector, Kinematic, Seek

def main():
    while True:
        try:
            response = requests.get("http://localhost:5000/get_data")
            if response.status_code == 200:
                data = response.json()["data"]
                print("Data from server.py:", data)

                # 데이터를 변수로 추출
                time_value = data["time"]
                distance = data["distance"]
                
                player_pos_x = data["playerPos"]["x"]
                player_pos_y = data["playerPos"]["y"]
                player_pos_z = data["playerPos"]["z"]
                player_speed = data["playerSpeed"]
                player_health = data["playerHealth"]
                player_turret_x = data["playerTurretX"]
                player_turret_y = data["playerTurretY"]
                player_body_x = data["playerBodyX"]
                player_body_y = data["playerBodyY"]
                
                enemy_pos_x = data["enemyPos"]["x"]
                enemy_pos_y = data["enemyPos"]["y"]
                enemy_pos_z = data["enemyPos"]["z"]
                enemy_speed = data["enemySpeed"]
                enemy_health = data["enemyHealth"]
                enemy_turret_x = data["enemyTurretX"]
                enemy_turret_y = data["enemyTurretY"]
                enemy_body_x = data["enemyBodyX"]
                enemy_body_y = data["enemyBodyY"]

                print(f"Time: {time_value}")
                print(f"Distance: {distance}")
                print(f"Player Position: ({player_pos_x}, {player_pos_y}, {player_pos_z})")
                print(f"Enemy Position: ({enemy_pos_x}, {enemy_pos_y}, {enemy_pos_z})")
                print(f"Player Health: {player_health}, Enemy Health: {enemy_health}")

                # 거리 계산 확인
                calculated_distance = math.sqrt(
                    (player_pos_x - enemy_pos_x) ** 2 +
                    (player_pos_z - enemy_pos_z) ** 2
                )
                print(f"Calculated Distance: {calculated_distance}")

            else:
                print("No data available in server.py yet.")
        except requests.exceptions.RequestException as e:
            print("Error fetching data:", e)
        time.sleep(2)

if __name__ == "__main__":
    character = Kinematic(position=Vector(0, 0), orientation=0.0)
    target = Kinematic(position=Vector(5, 5), orientation=0.0)
    seek = Seek(character, target, maxAcceleration=1.0)
    steering = seek.getSteering()
    print("동적 Seek 행동:")
    print(f"선형 가속도: {steering.linear}, 각가속도: {steering.angular:.2f}")
    character.update(steering, maxSpeed=2.0, time=1.0)
    print(f"업데이트 후 위치: {character.position}, 속도: {character.velocity}, 방향: {character.orientation:.2f}")
    main()
    

In [ ]:
import time
import requests
import math
from gameAI import Kinematic, Vector, Seek
import threading

class GameState:
    """게임 데이터를 저장하고 관리하는 클래스"""
    def __init__(self):
        self.time_value = None
        self.distance = None
        self.player_pos = {"x": 0, "y": 0, "z": 0}
        self.player_speed = 0.0
        self.player_health = 0.0
        self.player_turret_x = 0.0
        self.player_turret_y = 0.0
        self.player_body_x = 0.0
        self.player_body_y = 0.0
        self.enemy_pos = {"x": 0, "y": 0, "z": 0}
        self.enemy_speed = 0.0
        self.enemy_health = 0.0
        self.enemy_turret_x = 0.0
        self.enemy_turret_y = 0.0
        self.enemy_body_x = 0.0
        self.enemy_body_y = 0.0

    def update(self, data):
        """수신된 데이터를 기반으로 상태를 업데이트"""
        self.time_value = data["time"]
        self.distance = data["distance"]
        
        self.player_pos["x"] = data["playerPos"]["x"]
        self.player_pos["y"] = data["playerPos"]["y"]
        self.player_pos["z"] = data["playerPos"]["z"]
        self.player_speed = data["playerSpeed"]
        self.player_health = data["playerHealth"]
        self.player_turret_x = data["playerTurretX"]
        self.player_turret_y = data["playerTurretY"]
        self.player_body_x = data["playerBodyX"]
        self.player_body_y = data["playerBodyY"]
        
        self.enemy_pos["x"] = data["enemyPos"]["x"]
        self.enemy_pos["y"] = data["enemyPos"]["y"]
        self.enemy_pos["z"] = data["enemyPos"]["z"]
        self.enemy_speed = data["enemySpeed"]
        self.enemy_health = data["enemyHealth"]
        self.enemy_turret_x = data["enemyTurretX"]
        self.enemy_turret_y = data["enemyTurretY"]
        self.enemy_body_x = data["enemyBodyX"]
        self.enemy_body_y = data["enemyBodyY"]

    def __str__(self):
        """상태를 문자열로 반환 (디버깅용)"""
        return (f"Time: {self.time_value}, Distance: {self.distance}, "
                f"Player Pos: ({self.player_pos['x']}, {self.player_pos['z']}), "
                f"Enemy Pos: ({self.enemy_pos['x']}, {self.enemy_pos['z']})")

class GameServer:
    """게임 서버 로직을 실행하는 클래스"""
    def __init__(self):
        self.state = GameState()
        self.character = Kinematic(position=Vector(0, 0), orientation=0.0)
        self.target = Kinematic(position=Vector(5, 5), orientation=0.0)
        self.seek = Seek(self.character, self.target, maxAcceleration=1.0)

    def fetch_data(self):
        """서버에서 데이터를 가져와 상태를 업데이트"""
        try:
            response = requests.get("http://localhost:5000/get_data")
            if response.status_code == 200:
                data = response.json()["data"]
                self.state.update(data)
                print("Data from server.py:", data)
                return True
            else:
                print("No data available in server.py yet.")
                return False
        except requests.exceptions.RequestException as e:
            print("Error fetching data:", e)
            return False

    def run(self):
        """메인 루프 실행"""
        while True:
            if self.fetch_data():
                # 상태 사용 예시
                print(self.state)

                # Seek 행동 예시 (time_value 사용)
                steering = self.seek.getSteering()
                self.character.update(steering, maxSpeed=2.0, time=self.state.time_value)
                print(f"Character Position: {self.character.position}, "
                      f"Velocity: {self.character.velocity}, "
                      f"Orientation: {self.character.orientation:.2f}")
            time.sleep(1)

if __name__ == "__main__":
    # GameServer 인스턴스 생성 및 실행
    server = GameServer()
    thread = threading.Thread(target=server.run, daemon=True)
    thread.start()

    # 외부에서 state 접근 예시 (지속 실행)
    while True:
        if server.state.time_value is not None:
            print(f"Time outside class: {server.state.time_value}")
        else:
            print("Waiting for time_value...")
        time.sleep(1)

In [ ]:
# server.py
import math
import requests
import time
import threading
from gameAI import Kinematic, Vector, Seek
from utills import sharedKeyValue

class GameState:
    """게임 데이터를 저장하고 관리하는 클래스"""
    def __init__(self):
        self.time_value = None
        self.distance = None
        self.key = None
        self.player_pos = {"x": 0, "y": 0, "z": 0}
        self.player_speed = 0.0
        self.player_health = 0.0
        self.player_turret_x = 0.0
        self.player_turret_y = 0.0
        self.player_body_x = 0.0
        self.player_body_y = 0.0
        self.enemy_pos = {"x": 0, "y": 0, "z": 0}
        self.enemy_speed = 0.0
        self.enemy_health = 0.0
        self.enemy_turret_x = 0.0
        self.enemy_turret_y = 0.0
        self.enemy_body_x = 0.0
        self.enemy_body_y = 0.0

    def updateData(self, data):
        """수신된 데이터를 기반으로 상태를 업데이트"""
        self.time_value = data["time"]
        self.distance = data["distance"]
        
        self.player_pos["x"] = data["playerPos"]["x"]
        self.player_pos["y"] = data["playerPos"]["y"]
        self.player_pos["z"] = data["playerPos"]["z"]
        self.player_speed = data["playerSpeed"]
        self.player_health = data["playerHealth"]
        self.player_turret_x = data["playerTurretX"]
        self.player_turret_y = data["playerTurretY"]
        self.player_body_x = data["playerBodyX"]
        self.player_body_y = data["playerBodyY"]
        
        self.enemy_pos["x"] = data["enemyPos"]["x"]
        self.enemy_pos["y"] = data["enemyPos"]["y"]
        self.enemy_pos["z"] = data["enemyPos"]["z"]
        self.enemy_speed = data["enemySpeed"]
        self.enemy_health = data["enemyHealth"]
        self.enemy_turret_x = data["enemyTurretX"]
        self.enemy_turret_y = data["enemyTurretY"]
        self.enemy_body_x = data["enemyBodyX"]
        self.enemy_body_y = data["enemyBodyY"]

    def updatekey(self, key):
        self.key = key

    def __str__(self):
        """상태를 문자열로 반환 (디버깅용)"""
        return (f"Time: {self.time_value}, Distance: {self.distance}, "
                f"Player Pos: ({self.player_pos['x']}, {self.player_pos['z']}), "
                f"Enemy Pos: ({self.enemy_pos['x']}, {self.enemy_pos['z']})")

class GameServer:
    """게임 서버 로직을 실행하는 클래스"""
    def __init__(self):
        self.state = GameState()
        # 캐릭터 초기 위치를 플레이어 위치로 설정하거나 (0, 0)으로 시작
        self.character = Kinematic(position=Vector(0, 0), orientation=0.0)
        # 타겟은 적 위치로 동적 업데이트 예정이므로 초기값은 임의 설정
        self.target = Kinematic(position=Vector(0, 0), orientation=0.0)
        self.seek = Seek(self.character, self.target, maxAcceleration=1.0)

    def fetch_data(self):
        """서버에서 데이터를 가져와 상태를 업데이트"""
        try:
            response_data = requests.get("http://localhost:5000/get_data")
            if response_data.status_code == 200:
                data = response_data.json()["data"]
                self.state.updateData(data)
                # 적 위치로 타겟 업데이트
                self.target.position = Vector(self.state.enemy_pos["x"], self.state.enemy_pos["z"])
                print("Data from server.py:", data)
                return True
            else:
                print("No data available in server.py yet.")
                return False
        except requests.exceptions.RequestException as e:
            print("Error fetching data:", e)
            return False

    def run(self):
        """메인 루프 실행"""
        while True:
            if self.fetch_data():
                # 상태 사용 예시
                print(self.state)

                # Seek 행동: 적 위치를 향해 이동
                steering = self.seek.getSteering()
                # 시간 간격을 현실적으로 조정 (예: 1초 대신 0.016초 = 60 FPS)
                self.character.update(steering, maxSpeed=2.0, time=0.016)
                print(f"Character Position: {self.character.position}, "
                      f"Velocity: {self.character.velocity}, "
                      f"Orientation: {self.character.orientation:.2f}")
            time.sleep(1)

if __name__ == "__main__":
    # GameServer 인스턴스 생성 및 실행
    server = GameServer()
    thread = threading.Thread(target=server.run, daemon=True)
    thread.start()

    while True:
        if server.state.time_value is not None:
            print(f"Time outside class: {server.state.time_value}")
        else:
            print("Waiting for time_value...")
        time.sleep(1)